In [1]:
import pandas as pd
import json
import numpy
from pandas.io.json import json_normalize
from scipy.spatial.distance import cosine
import csv
# from pyspark import SparkContext
# from pyspark import SparkConf
# from pyspark.sql import SQLContext
from pyspark.sql.functions import udf #user defined function
from pyspark.sql.functions import lit, col
from pyspark.sql.types import *
from pyspark.sql import HiveContext
import ast

In [2]:
# Not needed for Spark on EC2
# sc = SparkContext("local", "Region Network")

In [7]:
# loading the raw data
# df = pd.read_csv('../../data/CDR/hash/sample.csv') 
# df.columns = ['index','time','source','dest','call']
sqlCtx = SQLContext(sc)
# cs_df = sqlCtx.createDataFrame(df)

cs_df = sc.textFile("s3n://census-cdr/mi-to-mi/*")

In [10]:
type(cs_df.collect())

KeyboardInterrupt: 

In [ ]:
cs_df.take(5)

In [2]:

# loading the region-cell data
table = pd.read_csv('../../data/CDR/hash/intersect.csv', header = None) 
table.columns = ['region', 'proportions']
table.index = table.region
table.sort_values(['region'], inplace=True)

# loading the cell-proportion data
prop_table = pd.read_csv('../../data/CDR/hash/cell_intersect.csv', header = None) 
prop_table.columns = ['cell', 'proportions']
prop_table.index = prop_table.cell
prop_table.sort_values(['cell'], inplace=True)


In [39]:


def get_cells_per_region(table, region_id):
    ids = table.iloc[region_id].proportions
    ids = ast.literal_eval(table.get_value(region_id, "proportions"))
    return ids.keys()

def get_call_data(source, dest):        
    source_dict = get_cells_per_region(table, source)
    dest_dict = get_cells_per_region(table, dest)
    query = "SELECT * from cs_df WHERE "
    
    cs_df.registerTempTable("cs_df")
    i = 1    
    
    for skey in source_dict:                  
        query += "source = " + str(skey) 
        if len(source_dict) > i:
            query += " OR "
        i += 1
    
    subset = sqlCtx.sql(query)
    subset.registerTempTable("subset")
    
    i=1
    query = "SELECT * from subset WHERE "
            
    for dkey in dest_dict:  
        query += "dest = " + str(dkey)
        if len(dest_dict) > i:
            query += " OR "
        i += 1
        
    subset2 = sqlCtx.sql(query)
            
    return subset2

def calculate_actual_call(s_cell, d_cell, call, s_region, d_region):
    """
        Create another column on the subset DataFrame that is proportional to the regions.
    """
    source_prop = ast.literal_eval(prop_table.get_value(s_cell, "proportions"))
    dest_prop = ast.literal_eval(prop_table.get_value(d_cell, "proportions"))

    try:
        final = source_prop[str(s_region)] * dest_prop[str(d_region)] * call
    except:
        final = 0
    
    return final

In [121]:

schema = StructType([
            StructField("time", IntegerType(), nullable=False),
            StructField("adjusted_call", FloatType(), nullable=False),
            StructField("source_region", IntegerType(), nullable=False),
            StructField("dest_region", IntegerType(), nullable=False)            
    ])

region_network = sqlCtx.createDataFrame([], schema)
udf_calls = udf(calculate_actual_call, FloatType())

for s in range(53,81):
    for d in range(53,55):
        # get a subset of records for the source and dest
        subdf = get_call_data(s, d)        
        subdf = subdf.withColumn("source_region", lit(s))
        subdf = subdf.withColumn("dest_region", lit(d))
        print (s, d)
        # create a column with adjusted call values
        try:
            subdf = subdf.select("time","source", "dest", "call", "source_region", "dest_region", udf_calls("source", "dest", "call", "source_region", "dest_region").alias("adjusted_call"))
            # subdf = subdf.withColumn("adjusted_call", udf_calls("source", "dest", "call", "source_region", "dest_region"))
        except:
            continue
        
        # do aggregation for 
        subdf = subdf.groupBy("time").agg({                
                "source_region": "max",
                "dest_region": "max",
                "adjusted_call": "sum"
            })
        region_network = region_network.unionAll(subdf)
    break
        
region_network.show()
region_network.toPandas().to_csv('../../data/CDR/generated/region-network.csv')

(53, 53)
(53, 54)
time          adjusted_call        source_region dest_region
1383329400000 0.011526255459784807 53            53         
1383325800000 0.001481413943068... 53            53         
1383315000000 0.015213906118333398 53            53         
1383301800000 0.010277155626681633 53            53         
1383262200000 0.021567917523647395 53            53         
1383340200000 0.006967352158312... 53            53         
1383319800000 0.020507195686150226 53            53         
1383305400000 0.012868893607446807 53            53         
1383297000000 0.07499233187604659  53            53         
1383324600000 0.012148591918048623 53            53         
1383345000000 0.015190431874657406 53            53         
1383321000000 0.025399163309145933 53            53         
1383295800000 0.02217467739683343  53            53         
1383339000000 0.021943406006812438 53            53         
1383300600000 0.05813425869018829  53            53         
138330

In [122]:
subdf.show()

time          SUM(adjusted_call)   MAX(source_region) MAX(dest_region)
1383329400000 5.523878759277068E-4 53                 54              
1383305400000 0.023464756045200375 53                 54              
1383334200000 0.003130371697125156 53                 54              
1383318000000 4.113162091812228E-5 53                 54              
1383322800000 5.523878759277068E-4 53                 54              
1383294000000 0.017197831941302866 53                 54              
1383323400000 0.003130371697125156 53                 54              
1383312600000 1.840541699493769... 53                 54              
1383336600000 0.018166079535149038 53                 54              
1383303000000 3.610459698393242... 53                 54              
1383326400000 0.018166079535149038 53                 54              
1383310800000 4.113162091812228E-5 53                 54              
1383315600000 0.011834605131298304 53                 54              
138330